In [1]:
# load densenet arch and make an eye detector (idea: with attention regions)

In [2]:
import os
import sys

sys.path.append('../')

In [3]:
from tflow.DenseNet10.keras_fc_densenet import _create_fc_dense_net
from tflow.DenseNet10.keras_fc_densenet import _dense_block, _transition_down_block

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Dense, Input, BatchNormalization, Activation, Dropout, GlobalMaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.regularizers import l2

In [5]:
img_input = Input((224, 224, 3))
bn_momentum = 0.9
concat_axis = -1
data_format = "channels_last"
dropout_rate = 0.2
weight_decay = 1e-4
transition_pooling = 'max'

# Initial convolution
with tf.name_scope("Initial"):
    x = Conv2D(
        48,
        (3, 3),
        kernel_initializer="he_normal",
        padding="same",
        use_bias=False,
        kernel_regularizer=l2(weight_decay),
        data_format=data_format,
    )(img_input)
    x = BatchNormalization(
        momentum=bn_momentum, axis=concat_axis, epsilon=1.1e-5
    )(x)
    x = Activation("relu")(x)

2022-07-26 00:04:01.904802: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 00:04:01.931294: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 00:04:01.931697: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 00:04:01.932203: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

In [6]:
# keeps track of the current number of feature maps
nb_filter = 42
growth_rate = 4
nb_dense_block = 4
nb_layers_per_block = 3
bottleneck_nb_layers = nb_layers_per_block
nb_layers = [nb_layers_per_block] * (2 * nb_dense_block + 1)

# collect skip connections on the downsampling path so that
# they can be concatenated with outputs on the upsampling path
skip_list = []

# Build the downsampling path by adding dense blocks and transition down blocks
for block_idx in range(nb_dense_block):
    x, nb_filter, _ = _dense_block(
        x,
        nb_layers[block_idx],
        nb_filter,
        growth_rate,
        bn_momentum=bn_momentum,
        dropout_rate=dropout_rate,
        data_format=data_format,
        block_prefix="DenseBlock_%i" % block_idx,
    )

    skip_list.append(x)
    x = _transition_down_block(
        x,
        nb_filter,
        weight_decay=weight_decay,
        bn_momentum=bn_momentum,
        transition_pooling=transition_pooling,
        data_format=data_format,
        block_prefix="TransitionDown_%i" % block_idx,
    )

x = GlobalAveragePooling2D()(x)
x = Dense(64)(x)
x = Dense(4)(x)

In [7]:
model = Model(inputs=img_input, outputs=x)

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 224, 224, 48  1296        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 224, 224, 48  192        ['conv2d[0][0]']                 
 alization)                     )                                                             